# 本筆記的目的：了解如何以UNet來分割影像。

此範例使用[Carvana Image Masking Challenge](https://www.kaggle.com/c/carvana-image-masking-challenge)所提供的資料集。

---

# 索引

1. [準備資料](#1.-準備資料)
2. [檢視資料集](#2.-檢視資料集)
3. [訓練模型](#3.-訓練模型)
4. [拿訓練好的模型做預測](#4.-拿訓練好的模型做預測)
5. [後記](#5.-後記)
---

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from keras.optimizers import Adam
from keras import backend as K

from sklearn.model_selection import train_test_split

import os

from unet import get_unet, UNet_utils

## 1. 準備資料

In [ ]:
# 給予路徑資訊
data_dir = "../datasets/kaggle-car-segmentation/train/"        # 圖路徑
mask_dir = "../datasets/kaggle-car-segmentation/train_masks/"  # 遮罩路徑

all_images = os.listdir(data_dir)
print("number of images =", len(all_images) )                  # 印出圖片數

# 將資料切分為 訓練資料(用於訓練模型) & 驗證資料(用於驗證模型)
train_images, validation_images = train_test_split(all_images, train_size=0.8, test_size=0.2)

[[返回索引]](#索引)

## 2. 檢視資料集

In [ ]:
# 得到一個generator，每次可得出8筆資料。
batch_size = 8
train_gen = UNet_utils.data_gen_small(data_dir, mask_dir, train_images, batch_size, (128, 128) )

In [ ]:
images, masks = next(train_gen)                 # 從generator撈出8筆資料。
                                                # 資料分別是圖，以及其相對應的遮罩。
assert len(images) == len(masks) == batch_size  # 確定撈出來的資料真的是有8筆。

# 檢視資料：從撈出來的資料中，畫出五張圖與其相應遮罩。
fig,axes = plt.subplots(5,2,figsize=(10,20))

for idx,(image,mask) in enumerate( zip(images,masks) ):
    axes[idx,0].imshow(image)
    axes[idx,1].imshow( UNet_utils.grey2rgb(mask), alpha=0.5 )
    
    axes[idx,0].axis('off')
    axes[idx,1].axis('off')
    
    if idx ==4:
        break

# plt.imshow(img[0])
# plt.imshow(grey2rgb(msk[0]), alpha=0.5)

[[返回索引]](#索引)

## 3. 訓練模型

In [ ]:
# 取得UNet模型
model = get_unet()
# 看一下模型摘要
#model.summary()

In [ ]:
def dice_metric(y_true, y_pred, smooth = 1.E-6):
    '''定義Dice metric。'''
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    
    return 2.*intersection  / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    '''定義Dice loss。
       註：Dice metric越高，代表mask學的越好。因此，我們的目標是讓機器去嘗試最大化Dice metric。
    '''
    return -dice_metric(y_true, y_pred)

# 訓練模型
model.compile(optimizer=Adam(1e-4), loss=dice_loss, metrics=[dice_metric])
train_record = model.fit_generator(train_gen, steps_per_epoch=100, epochs=5)

# 檢視模型訓練情形
plt.plot(train_record.history['dice_metric'],ms=5,marker='o',label='dice metric')
plt.plot(train_record.history['loss'],ms=5,marker='o',label='loss')
plt.xlabel("Epoch")
plt.legend()
plt.show()

[[返回索引]](#索引)

## 4. 拿訓練好的模型做預測

In [ ]:
# 得到可以產生驗證資料的generator。
batch_size = 8
val_gen = UNet_utils.data_gen_small(data_dir, mask_dir, train_images, batch_size, (128, 128) )

images, masks = next(val_gen)                   # 從generator撈出8筆資料來看一下。
                                                # 資料分別是圖，以及其相對應的遮罩。
assert len(images) == len(masks) == batch_size  # 確定撈出來的資料真的是有8筆。

mask_preds = model.predict(images)              # 拿剛建好的模型去預測圖片所對應的遮罩樣貌。

# 檢視資料：畫出五張圖，相應遮罩(真實)，相應遮罩(預測)。
fig,axes = plt.subplots(5,3,figsize=(15,20))
fig.suptitle("Left to Right: image, mask (ground truth), mask (prediction)",fontsize="30",y=1.02)

for idx,(image,mask,mask_pred) in enumerate( zip(images,masks, mask_preds) ):

    axes[idx,0].imshow(image)
    axes[idx,1].imshow( UNet_utils.grey2rgb(mask), alpha=0.5 )
    axes[idx,2].imshow( UNet_utils.grey2rgb(mask_pred), alpha=0.5 )
    
    axes[idx,0].axis('off')
    axes[idx,1].axis('off')
    axes[idx,2].axis('off')
    
    if idx ==4:
        break
plt.tight_layout()

[[返回索引]](#索引)

---

## 5. 後記

1. 我們於訓練模型的時候，並沒有使用到驗證資料(validation data)來做驗證。請嘗試將驗證資料餵給模型去做驗證。
2. 我們的資料相當理想，因為汽車大小都差不多。然而，實際在路上拍攝時，因為鏡頭和汽車的距離不一定，車的大小也會不太一樣。且於白天，黑夜，下雨，下雪，不同的環境下，攝影機拍出來的影像看起來會不一樣。若需要模型比較能夠應付各種不同的情況，我們得需要擁有更豐富的訓練資料，又或者，於建模時，嘗試使用*資料增益* (*data augmentation*)。
3. 我們自定義的generator速度太慢了，GPU工作的很沒效率，因為它一直在等generator從硬碟載入資料給它。解決方案：
    1. 把所有資料載入至電腦的RAM，然後從RAM去載入資料(會比從硬碟載入還要快很多)至GPU。
    2. 做出一個generator。該generator會以*multi-threading*或*multi-processing*的方式，從硬碟將資料載入至RAM。若資料非常大，無法將其全部放置於RAM，那麼，我們會希望把資料放置於一個固定大小的*queue*(於RAM裡面)。這個*queue*必須隨時處於滿載的情況，這樣，GPU需要資料的時候，就可以即時的去那邊提取資料。


[[返回索引]](#索引)